# Ejemplo Ventanas 3
Este ejemplo es similar a los anteriores, pero en este caso usamos datos de bolsa.

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split
import string

spark = SparkSession.builder \
    .master("spark://spark-master:7077") \
    .appName("ejemplo_ventanas_3") \
    .config("spark.sql.legacy.timeParserPolicy", "LEGACY") \
    .config("spark.eventLog.enabled", "true") \
    .config("spark.eventLog.dir", "hdfs:///spark/logs/history") \
    .config("spark.history.fs.logDirectory", "hdfs:///spark/logs/history") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


Usaremos el archivo **bolsa1.json**. Lo primero es crear un esquema apropiado para dicho archivo. 

In [2]:
# Definimos el esquema de los datos de entrada
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
bolsaSchema = StructType([
    StructField("CreatedTime", StringType()),
    StructField("Type", StringType()),
    StructField("Amount", IntegerType()),
    StructField("BrokerCode", StringType())
])

Debe  colocarse en el directorio **/user/jovyan/entrada**. Si se coloca en otro hay que modificar la opción *path* de *readStream*.

In [3]:
# Configuramos la lectura de fichero en formato JSON
rawDF = spark.readStream \
        .format("json") \
        .option("path", "entrada") \
        .option("maxFilesPerTrigger", 1) \
        .schema(bolsaSchema) \
        .load()

rawDF.printSchema()

root
 |-- CreatedTime: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Amount: integer (nullable = true)
 |-- BrokerCode: string (nullable = true)



Creamos un nuevo DF a partir del anterior. A las columnas existentes añadiremos dos nuevas:
- **Compras**: tendrá el valor de la columna *Amount* si la columna *Type* tiene el valor *BUY*, 0 en caso contrario.
- **Ventas**: tendrá el valor de la columna *Amount* si la columna *Type* tiene el valor *SELL*, 0 en caso contrario.

In [4]:
from pyspark.sql.functions import to_timestamp, col, expr
accionesDF = rawDF.withColumn("CreatedTime", to_timestamp(col("CreatedTime"), "yyyy-MM-dd HH:mm:ss")) \
    .withColumn("Compras", expr("case when Type == 'BUY' then Amount else 0 end")) \
    .withColumn("Ventas", expr("case when Type == 'SELL' then Amount else 0 end"))

accionesDF.printSchema()

root
 |-- CreatedTime: timestamp (nullable = true)
 |-- Type: string (nullable = true)
 |-- Amount: integer (nullable = true)
 |-- BrokerCode: string (nullable = true)
 |-- Compras: integer (nullable = true)
 |-- Ventas: integer (nullable = true)



Agrupamos por ventanas de **CreatedTime** de 15 minutos de duración, agregando mediante suma las columnas *compras* y *ventas*. 

In [5]:
from pyspark.sql.functions import window, sum
windowDF = accionesDF \
    .groupBy(  # col("BrokerCode"),
         window(col("CreatedTime"), "15 minutes")) \
    .agg(sum("Compras").alias("Compras"),
         sum("Ventas").alias("Ventas"))

salidaDF = windowDF.select("window.start", "window.end", "Compras", "Ventas")

Por último, iniciamos el procesamiento en streaming indicando la salida a consola.

In [ ]:


bolsaWriterQuery = salidaDF.writeStream \
    .format("console") \
    .outputMode("complete") \
    .start()

bolsaWriterQuery.awaitTermination()

Hay una serie de limitaciones: 
- Al usar agregaciones en operaciones con ventana no podemos usar algunos modos de salida, como ya se habisto en el notebook 00. En concreto, no se pueden usar los modos *update* y *append*.
- Para poder usar el *sink* tipo archivo es necesario el modo *append*.
- Para poder usar el modo *append* es necesario aplicar el concepto de *watermarking* que se verá en el notebook 04.d.

In [6]:
# Ejemplo: No se puede usar el modo append sin watermark.
bolsaWriterQuery = salidaDF.writeStream \
    .format("parquet") \
    .queryName("BolsaWQuery") \
    .outputMode("append") \
    .option("path", "salida") \
    .option("checkpointLocation", "chk-point-dir-caso7") \
    .trigger(processingTime="1 minute") \
    .start()

AnalysisException: Data source parquet does not support Update output mode.